## **Análise de série de vazões (Xingó)**

### 1 - Leitura dos dados e gerando hidrogramas

In [29]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go


import sys
sys.path.insert(0, r'C:\Users\arist\OneDrive\Documentos\UFAL\PIBIC 24-25\lib_clebson\HydroComp')

from files.ons import Ons
from files.ana import Ana
from series.flow import Flow
from series.chuva import Chuva

In [2]:
file = "ONS_daily_flow.csv"
dados = pd.read_csv(file, index_col=0, parse_dates=True)
dados = pd.DataFrame(dados["XINGO (178)"])

#Convetendo obj pandas para Series Flow
dados = Flow(pd.DataFrame(dados["XINGO (178)"]))

#Gerando hidrograma
fig, data = dados.plot_hydrogram()
#py.offline.plot(fig, filename='gráficos/hidrograma.html')
pio.show(fig)

#Calculando picos anuais
peaks_max = dados.maximum(station="XINGO (178)")

#Gerando hidrograma destanco os valores de pico
fig, data = peaks_max.plot_hydrogram()
#py.offline.plot(fig, filename='gráficos/histo_max.html')
pio.show(fig)


### **2 - Séries de durações parciais**

#### 2.1 - Separando eventos hidrológicos e calculando métricas

In [3]:
station = "XINGO (178)"

#'stationary' or 'events_by_year'
type_threshold = 'stationary' 

#'flood' or 'drought'
type_event = 'flood'

#'media', 'mediana' or 'autocorrelation' 
type_criterion = 'median'

#Peaks or percentil
value_threshold = 0.75

peaks_sdp = dados.parcial(station, type_threshold, type_event, type_criterion, value_threshold)
peaks_sdp.peaks.head()

,Duration,Start,End,peaks
1931-05-06,143,1931-01-01,1931-05-24,7527.0
1932-02-09,61,1932-01-18,1932-03-19,5493.0
1933-02-20,119,1932-11-29,1933-03-28,6476.0
1934-02-07,76,1933-12-26,1934-03-12,6994.0
1935-02-24,134,1935-01-08,1935-05-22,7217.0


In [4]:
#Armazenando dataframe da SDP calculada
df_sdp = peaks_sdp.peaks

In [5]:
fig, data = peaks_sdp.plot_hydrogram(title='SDP-Xingo')
py.offline.plot(fig, filename='gráficos/histo_sdp_by_year.html')

'gráficos/histo_sdp_by_year.html'

In [34]:
peaks_sdp.mvs()
peaks_sdp.fit[0]

-0.16327398505263607

### **3 - Modelagem das distribuições marginais**

#### **3.1 - Magnitude (Generalizada de Pareto)**

##### Máxima Verossimilhança

In [70]:
#Distribuição cumulativa
title = 'Xingó'
type_function = 'cumulative'
fig_acum_mvs_pareto, data_acum_mvs_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mvs')
#py.offline.plot(fig, filename='gráficos/distrib_cum_sdp_mvs.html')
#pio.show(fig_acum_mvs_pareto)

#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'density'
fig_density_mvs_pareto, data_density_mvs_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mvs')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
#pio.show(fig_density_mvs_pareto)

##### Momemtos L

In [34]:
#Distribuição cumulativa
title = 'Xingó'
type_function = 'cumulative'
fig_acum_mml_pareto, data_acum_mml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml')
#py.offline.plot(fig, filename='gráficos/distrib_cum_sdp_mvs.html')
pio.show(fig_acum_mml_pareto)

#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'density'
fig_density_mml_pareto, data_density_mmml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
pio.show(fig_density_mml_pareto)

##### Verificando parâmetros

In [37]:
#Parâmetros 
print("MML: ", peaks_sdp.mml())

print("MVS", peaks_sdp.mvs())

MML:  [-0.006753770914668085, 4502.188466580488, 2973.836978393338]
MVS (-0.16327398505263607, 3881.9241208697904, 4122.4799673721955)


##### Visualizando distribuições pelos dois estimadores

In [72]:
#Definindo legenda
fig_density_mml_pareto['data'][0].name = "MML"
fig_density_mvs_pareto['data'][0].name = "MVS"

#Ajustando cor dos gráficos
fig_density_mml_pareto['data'][0].line.color = 'rgb(0, 0, 0)'

#Densidade
fig_density = go.Figure(data = fig_density_mvs_pareto['data'] + fig_density_mml_pareto['data'])

fig_density.update_layout(
    title="GP - Densidade Xingó",
    xaxis_title="Vazão (m³/s)",
    showlegend=True,
    font=dict(family='Time New Roman', size=20, color='rgb(0,0,0)')) 

fig_density.show()


#Acumulado

#Definindo legenda
fig_acum_mml_pareto['data'][0].name = "MML"
fig_acum_mvs_pareto['data'][0].name = "MVS"

#Ajustando cor dos gráficos
fig_acum_mml_pareto['data'][0].line.color = 'rgb(0, 0, 0)'

fig_acum = go.Figure(data = fig_acum_mvs_pareto['data'] + fig_acum_mml_pareto['data'])

fig_acum.update_layout(
    title="GP - Densidade Xingó",
    xaxis_title="Vazão (m³/s)",
    showlegend=True,
    font=dict(family='Time New Roman', size=20, color='rgb(0,0,0)')) 

fig_acum.show()

#### 3.2 - Duração ( Pearson III) 

#### 3.3 - Período de ocorrência (Von-Mises)